# 15차시: [실습] 네이버 금융에서 뉴스 타이틀과 시장 지표 크롤링

## 학습 목표
- 네이버 금융 페이지의 HTML 구조를 분석하는 방법 학습
- 주요 시장 지표(환율, 유가, 지수)를 크롤링하는 실습
- 뉴스 헤드라인과 링크를 수집하는 실습
- 종목 재무정보를 pd.read_html()로 추출하는 방법 학습

## 학습 내용
1. 네이버 금융 페이지 구조 분석
2. 시장 지표 크롤링 (환율, 유가, 지수)
3. 뉴스 헤드라인 크롤링
4. 종목 재무정보 크롤링 (pd.read_html)
5. 크롤링 함수 작성


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime
import time
from IPython.display import display

In [2]:
# 공통 설정
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

def get_soup(url):
    """URL에서 BeautifulSoup 객체 반환"""
    response = requests.get(url, headers=HEADERS, timeout=10)
    response.raise_for_status()
    return BeautifulSoup(response.text, 'html.parser')

---
## 1. 네이버 금융 페이지 구조 분석

네이버 금융 메인 페이지: https://finance.naver.com/

### 주요 섹션
- **시세 정보**: 코스피, 코스닥, 환율, 유가 등
- **뉴스**: 증권 뉴스 헤드라인
- **인기 종목**: 거래량/상승률 상위 종목

---
## 2. 시장 지표 크롤링

### 2.1 KOSPI/KOSDAQ 지수

In [3]:
url = "https://finance.naver.com/sise/sise_index.naver?code=KOSPI"

# url = "https://finance.naver.com/sise/sise_index.naver?code=KOSDAQ"

soup = get_soup(url)

# 현재가
now_value = soup.select_one('#now_value')

# 증감 + 등락률
fluc = soup.select_one('#change_value_and_rate')
current = now_value.get_text(strip=True)

change = fluc.select_one('span').get_text(strip=True)
rate = fluc.get_text(strip=True).replace(change, '').replace('상승', '').replace('하락', '').strip()

print("현재가:", current)
print("증감:", change)
print("등락률:", rate)

현재가: 4,214.17
증감: 6.39
등락률: -0.15%


### 2.2 환율 크롤링

In [4]:
# 환율 크롤링
print("[환율 크롤링]")
print("=" * 60)

# 네이버 금융 환율 페이지
url = "https://finance.naver.com/marketindex/"
soup = get_soup(url)

exchange_data = []

# 등락 방향 추출 함수
def get_direction(item):
    head = item.select_one('.head_info')
    cls = head.get('class', []) if head else []
    return '상승' if 'point_up' in cls else ('하락' if 'point_dn' in cls else '보합')

# 환율 정보 추출
exchange_list = soup.select('#exchangeList li')

for item in exchange_list[:4]:  # 주요 4개만
    try:
        # 통화명
        name_elem = item.select_one('.h_lst .blind')
        name = name_elem.text.strip() if name_elem else 'N/A'

        # 현재가
        current_elem = item.select_one('.head_info .value')
        current = current_elem.text.strip() if current_elem else 'N/A'

        # 등락
        change_elem = item.select_one('.head_info .change')
        change = change_elem.text.strip() if change_elem else 'N/A'

        # 등락방향
        direction = get_direction(item)

        exchange_data.append({
            '통화': name,
            '환율': current,
            '등락': change,
            '등락방향': direction
        })
        print(f"{name}: {current} ({direction})")
    except Exception as e:
        continue

# DataFrame으로 정리
if exchange_data:
    df_exchange = pd.DataFrame(exchange_data)
    print("\n[환율 데이터]")
    display(df_exchange)

[환율 크롤링]
미국 USD: 1,447.00 (상승)
일본 JPY(100엔): 923.54 (상승)
유럽연합 EUR: 1,699.65 (상승)
중국 CNY: 207.38 (상승)

[환율 데이터]


,통화,환율,등락,등락방향
0,미국 USD,"1,447.00",6.00,상승
1,일본 JPY(100엔),923.54,2.45,상승
2,유럽연합 EUR,"1,699.65",6.84,상승
3,중국 CNY,207.38,1.28,상승


### 2.3 국제 유가 및 금 시세

In [5]:
# 유가 및 금 시세 크롤링
print("[국제 유가 및 금 시세]")
print("=" * 60)

# 네이버 금융 원자재 페이지
url = "https://finance.naver.com/marketindex/"
soup = get_soup(url)

commodity_data = []

# 등락 방향 추출 함수 (위에서 정의한 것과 동일)
def get_direction(item):
    head = item.select_one('.head_info')
    cls = head.get('class', []) if head else []
    return '상승' if 'point_up' in cls else ('하락' if 'point_dn' in cls else '보합')

# 원자재 시세 추출
commodity_items = soup.select('#oilGoldList li')

for item in commodity_items:
    try:
        name_elem = item.select_one('.h_lst .blind')
        name = name_elem.text.strip() if name_elem else 'N/A'

        current_elem = item.select_one('.head_info .value')
        current = current_elem.text.strip() if current_elem else 'N/A'

        change_elem = item.select_one('.head_info .change')
        change = change_elem.text.strip() if change_elem else 'N/A'

        # 등락방향
        direction = get_direction(item)

        commodity_data.append({
            '품목': name,
            '시세': current,
            '등락': change,
            '등락방향': direction
        })
        print(f"{name}: {current} ({direction})")
    except Exception as e:
        continue

# DataFrame으로 정리
if commodity_data:
    df_commodity = pd.DataFrame(commodity_data)
    print("\n[원자재 데이터]")
    display(df_commodity)

[국제 유가 및 금 시세]
WTI: 57.42 (하락)
휘발유: 1727.55 (하락)
국제 금: 4341.1 (하락)
국내 금: 201866.54 (하락)

[원자재 데이터]


,품목,시세,등락,등락방향
0,WTI,57.42,0.53,하락
1,휘발유,1727.55,1.27,하락
2,국제 금,4341.1,45.20,하락
3,국내 금,201866.54,991.71,하락


### 2.4 시장 지표 통합 함수

In [6]:
# 시장 지표 크롤링 통합 함수
def crawl_market_indicators():
    """
    네이버 금융에서 시장 지표를 크롤링하는 함수

    Returns:
        DataFrame: 시장 지표 데이터 (구분, 지표, 현재가, 등락, 등락방향, 수집시각)
    """
    url = "https://finance.naver.com/marketindex/"
    soup = get_soup(url)

    all_data = []
    crawl_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # 등락 방향 추출 함수 (point_up/point_dn 클래스 기반)
    def get_direction(item):
        head = item.select_one('.head_info')
        cls = head.get('class', []) if head else []
        return '상승' if 'point_up' in cls else ('하락' if 'point_dn' in cls else '보합')

    # 데이터 추가 함수
    def add_items(items, group, limit=None):
        for item in (items[:limit] if limit else items):
            try:
                name = item.select_one('.h_lst .blind')
                current = item.select_one('.head_info .value')
                change = item.select_one('.head_info .change')

                all_data.append({
                    '구분': group,
                    '지표': name.get_text(strip=True) if name else 'N/A',
                    '현재가': current.get_text(strip=True) if current else 'N/A',
                    '등락': change.get_text(strip=True) if change else 'N/A',
                    '등락방향': get_direction(item),
                    '수집시각': crawl_time
                })
            except:
                continue

    # 환율 데이터 (환전고시 환율 영역)
    exchange_items = soup.select('#exchangeList li')
    add_items(exchange_items, '환율', limit=4)

    # 원자재 데이터
    commodity_items = soup.select('#oilGoldList li')
    add_items(commodity_items, '원자재')

    return pd.DataFrame(all_data)

# 함수 테스트
print("[시장 지표 크롤링 함수 테스트]")
print("=" * 60)
df_market = crawl_market_indicators()
print(f"수집된 데이터: {len(df_market)}건\n")
df_market

[시장 지표 크롤링 함수 테스트]
수집된 데이터: 8건



,구분,지표,현재가,등락,등락방향,수집시각
0,환율,미국 USD,"1,447.00",6.00,상승,2026-01-01 11:54:41
1,환율,일본 JPY(100엔),923.54,2.45,상승,2026-01-01 11:54:41
2,환율,유럽연합 EUR,"1,699.65",6.84,상승,2026-01-01 11:54:41
3,환율,중국 CNY,207.38,1.28,상승,2026-01-01 11:54:41
4,원자재,WTI,57.42,0.53,하락,2026-01-01 11:54:41
5,원자재,휘발유,1727.55,1.27,하락,2026-01-01 11:54:41
6,원자재,국제 금,4341.1,45.20,하락,2026-01-01 11:54:41
7,원자재,국내 금,201866.54,991.71,하락,2026-01-01 11:54:41


---
## 3. 뉴스 헤드라인 크롤링

In [8]:
def crawl_financial_news(limit=10):
    """
    네이버 금융 주요 뉴스 헤드라인 크롤링

    Parameters
    ----------
    limit : int

    Returns
    -------
    pandas.DataFrame
        [제목, 출처, 링크, 수집시각]
    """

    url = "https://finance.naver.com/news/mainnews.naver"
    soup = get_soup(url)

    news_data = []
    crawl_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # 뉴스 리스트
    news_items = soup.select("ul.newsList li")

    for item in news_items[:limit]:
        try:
            # 제목 + 링크
            title_elem = item.select_one("dd.articleSubject a")
            if not title_elem:
                continue

            title = title_elem.get("title", title_elem.get_text(strip=True))
            link = title_elem.get("href", "")
            if link.startswith("/"):
                link = "https://finance.naver.com" + link

            # 언론사
            press_elem = item.select_one(".press")
            press = press_elem.get_text(strip=True) if press_elem else "N/A"

            news_data.append({
                "제목": title,
                "출처": press,
                "링크": link,
                "수집시각": crawl_time
            })

        except Exception:
            continue

    return pd.DataFrame(news_data)

In [9]:
from IPython.display import HTML, display

# 함수 테스트
print("[뉴스 크롤링 함수 테스트]")
print("=" * 60)
df_news_test = crawl_financial_news(limit=5)
print(f"수집된 뉴스: {len(df_news_test)}건\n")

if not df_news_test.empty:
    df_show = df_news_test.copy()

    # 제목을 클릭 가능한 링크(HTML)로 변환
    df_show["제목"] = df_show.apply(
        lambda row: f'<a href="{row["링크"]}" target="_blank">{row["제목"]}</a>',
        axis=1
    )

    # 링크 컬럼은 숨기고(원하면 유지 가능)
    df_show = df_show[["제목", "출처", "수집시각"]]

    # HTML로 표시 (escape=False 중요!)
    display(HTML(df_show.to_html(escape=False, index=False)))
else:
    print("❌ 수집된 뉴스가 없습니다.")

[뉴스 크롤링 함수 테스트]
수집된 뉴스: 5건



제목,출처,수집시각
"잘 나가는 방위산업株…한화에어로·LIG넥스원, 나란히 AA로 신용 ‘레벨업’ [투자360](휴일)",헤럴드경제,2026-01-01 11:54:55
"""11만전자·65만닉스는 시작""…줄줄이 '엄지척'",한국경제TV,2026-01-01 11:54:55
"올해도 코스피 파죽지세로…""상반기 5000 돌파""",MBN,2026-01-01 11:54:55
쿠팡 김범석·로저스 불출석·위증 혐의 고발…정부·국회 전방위 압박,MBN,2026-01-01 11:54:55
"""연금 방치 말고 굴리세요"" 금융데이터 분석해 ‘맞춤 포트폴리오’ [올해 눈여겨볼 재테크 트렌드]",파이낸셜뉴스,2026-01-01 11:54:55


---
## 4. 종목 재무정보 크롤링 (pd.read_html)

`pd.read_html()`은 HTML 테이블을 자동으로 DataFrame으로 변환합니다.

In [10]:
import pandas as pd
import numpy as np

def crawl_financial_data(stock_code: str) -> pd.DataFrame:
    url = f"https://finance.naver.com/item/main.nhn?code={stock_code}"
    try:
        tables = pd.read_html(url)

        # "재무표" 후보를 점수화해서 가장 그럴듯한 테이블 선택
        keywords = ("주요재무정보", "매출액", "영업이익", "당기순이익", "ROE", "EPS")
        def score(df):
            text = " ".join(map(str, df.columns)) + " " + " ".join(df.astype(str).head(3).values.ravel())
            return sum(k in text for k in keywords) + (df.shape[0] * df.shape[1]) / 1000

        df = max(tables, key=score)
        return df.replace("-", np.nan)

    except Exception as e:
        print(f"재무정보 추출 실패: {e}")
        return pd.DataFrame()

df_financial = crawl_financial_data("005930")
display(df_financial)

주요재무정보    최근 연간 실적                                       최근 분기 실적  \
       주요재무정보     2022.12     2023.12     2024.12  2025.12(E)    2024.09   
       주요재무정보      IFRS연결      IFRS연결      IFRS연결      IFRS연결     IFRS연결   
0         매출액  3022314.00  2589355.00  3008709.00  3282245.00  790987.00   
1        영업이익   433766.00    65670.00   327260.00   394567.00   91834.00   
2       당기순이익   556541.00   154871.00   344514.00   392120.00  101009.00   
3       영업이익률       14.35        2.54       10.88       12.02      11.61   
4        순이익률       18.41        5.98       11.45       11.95      12.77   
5   ROE(지배주주)       17.07        4.15        9.03        9.44       8.79   
6        부채비율       26.41       25.36       27.93         NaN      27.19   
7        당좌비율      211.68      189.46      187.80         NaN     190.56   
8         유보율    38144.29    39114.28    41772.84         NaN   41198.62   
9      EPS(원)     8057.00     2131.00     4950.00     5672.00    1440.00   
10     PER(배)        6.86       36.84       10.75       21.14      13.03   
11     BPS(원)    50817.00    52002.00    57981.00    63155.00   55376.00   
12     PBR(배)        1.09        1.51        0.92        1.90       1.11   
13   주당배당금(원)     1444.00     1444.00     1446.00     1524.00     361.00   
14   시가배당률(%)        2.61        1.84        2.72         NaN       0.59   
15    배당성향(%)       17.92       67.78       29.18         NaN      25.07   

                                                           
      2024.12    2025.03    2025.06    2025.09 2025.12(E)  
       IFRS연결     IFRS연결     IFRS연결     IFRS연결     IFRS연결  
0   757883.00  791405.00  745663.00  860617.00  886181.00  
1    64927.00   66853.00   46761.00  121661.00  160045.00  
2    77544.00   82229.00   51164.00  122257.00  151677.00  
3        8.57       8.45       6.27      14.14      18.06  
4       10.23      10.39       6.86      14.21      17.12  
5        9.03       9.24       7.95       8.37        NaN  
6       27.93      26.99      26.36      26.64        NaN  
7      187.80     187.68     190.87     204.62        NaN  
8    41772.84   42056.84   42340.19   43418.06        NaN  
9     1115.00    1186.00     733.00    1783.00    2202.00  
10      10.75      11.20      13.36      17.42      54.44  
11   57981.00   59059.00   58135.00   60632.00   63155.00  
12       0.92       0.98       1.03       1.38       1.90  
13     363.00     365.00     367.00     370.00        NaN  
14       0.68       0.63       0.61       0.44        NaN  
15      32.40      30.48      49.73      20.43        NaN

In [11]:
from datetime import datetime

# 종목 기본 정보 크롤링
def crawl_stock_info(stock_code: str) -> dict:
    url = f"https://finance.naver.com/item/main.nhn?code={stock_code}"
    soup = get_soup(url)

    info = {
        "종목코드": stock_code,
        "수집시각": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    }

    selectors = {
        "종목명": ".wrap_company h2 a",
        "현재가": ".no_today .blind",
        "전일비": ".no_exday .blind",
    }

    for key, css in selectors.items():
        elem = soup.select_one(css)
        info[key] = elem.text.strip() if elem else "N/A"

    return info

print("[종목 기본 정보 크롤링]")
print("=" * 60)
stock_info = crawl_stock_info("005930")
for key, value in stock_info.items():
    print(f"  {key}: {value}")

[종목 기본 정보 크롤링]
  종목코드: 005930
  수집시각: 2026-01-01 11:55:07
  종목명: 삼성전자
  현재가: 119,900
  전일비: 400


---
## 학습 정리

### 1. 네이버 금융 크롤링 URL
| 페이지 | URL |
|--------|-----|
| 메인 | https://finance.naver.com/ |
| 시장지표 | https://finance.naver.com/marketindex/ |
| 주요뉴스 | https://finance.naver.com/news/mainnews.naver |
| 종목 | https://finance.naver.com/item/main.nhn?code={종목코드} |

### 2. 핵심 크롤링 함수
```python
# 시장 지표 크롤링
df_market = crawl_market_indicators()

# 뉴스 헤드라인 크롤링
df_news = crawl_financial_news(limit=10)

# 종목 재무정보 크롤링
df_financial = crawl_financial_data("005930")
```

### 3. pd.read_html() 활용
```python
# HTML 테이블을 DataFrame으로 자동 변환
tables = pd.read_html(url)

# 특정 테이블 선택
df = tables[3]  # 인덱스로 선택
```

### 4. 크롤링 시 주의사항
- `time.sleep()`: 요청 간 딜레이로 서버 부하 방지
- `try-except`: 예외 처리로 안정적인 크롤링
- User-Agent 설정: 브라우저처럼 접근

---

### 다음 차시 예고
- 16차시: 크롤링 데이터 정제 및 SQLite 저장
  - Pandas로 데이터 정제
  - SQLite 데이터베이스 기초
  - 테이블 생성 및 데이터 저장/조회